Automatons, or Finite State Machines (FSM), are extremely useful to programmers when it comes to software design. You will be given a simplistic version of an FSM to code for a basic TCP session.

The outcome of this exercise will be to return the correct state of the TCP FSM based on the array of events given.

The input array of events will consist of one or more of the following strings:

APP_PASSIVE_OPEN, APP_ACTIVE_OPEN, APP_SEND, APP_CLOSE, APP_TIMEOUT, RCV_SYN, RCV_ACK, RCV_SYN_ACK, RCV_FIN, RCV_FIN_ACK

The states are as follows and should be returned in all capital letters as shown:

CLOSED, LISTEN, SYN_SENT, SYN_RCVD, ESTABLISHED, CLOSE_WAIT, LAST_ACK, FIN_WAIT_1, FIN_WAIT_2, CLOSING, TIME_WAIT

The input will be an array of events. Your job is to traverse the FSM as determined by the events, and return the proper state as a string, all caps, as shown above.

If an event is not applicable to the current state, your code will return "ERROR".
Action of each event upon each state:

(the format is INITIAL_STATE: EVENT -> NEW_STATE)

CLOSED: APP_PASSIVE_OPEN -> LISTEN
CLOSED: APP_ACTIVE_OPEN  -> SYN_SENT
LISTEN: RCV_SYN          -> SYN_RCVD
LISTEN: APP_SEND         -> SYN_SENT
LISTEN: APP_CLOSE        -> CLOSED
SYN_RCVD: APP_CLOSE      -> FIN_WAIT_1
SYN_RCVD: RCV_ACK        -> ESTABLISHED
SYN_SENT: RCV_SYN        -> SYN_RCVD
SYN_SENT: RCV_SYN_ACK    -> ESTABLISHED
SYN_SENT: APP_CLOSE      -> CLOSED
ESTABLISHED: APP_CLOSE   -> FIN_WAIT_1
ESTABLISHED: RCV_FIN     -> CLOSE_WAIT
FIN_WAIT_1: RCV_FIN      -> CLOSING
FIN_WAIT_1: RCV_FIN_ACK  -> TIME_WAIT
FIN_WAIT_1: RCV_ACK      -> FIN_WAIT_2
CLOSING: RCV_ACK         -> TIME_WAIT
FIN_WAIT_2: RCV_FIN      -> TIME_WAIT
TIME_WAIT: APP_TIMEOUT   -> CLOSED
CLOSE_WAIT: APP_CLOSE    -> LAST_ACK
LAST_ACK: RCV_ACK        -> CLOSED

"EFSM TCP"
# Examples

["APP_PASSIVE_OPEN", "APP_SEND", "RCV_SYN_ACK"] =>  "ESTABLISHED"

["APP_ACTIVE_OPEN"] =>  "SYN_SENT"

["APP_ACTIVE_OPEN", "RCV_SYN_ACK", "APP_CLOSE", "RCV_FIN_ACK", "RCV_ACK"] =>  "ERROR"

This kata is similar to Design a Simple Automaton (Finite State Machine), and you may wish to try that kata before tackling this one.

See wikipedia page Transmission Control Protocol for further details.

See http://www.medianet.kent.edu/techreports/TR2005-07-22-tcp-EFSM.pdf page 4, for the FSM diagram used for this kata.


In [18]:
def get_new_state(current_state, action):
    if current_state == 'CLOSED':
        if action == 'APP_PASSIVE_OPEN':
            return 'LISTEN'
        elif action == 'APP_ACTIVE_OPEN':
            return 'SYN_SENT'
        else:
            return 'ERROR'

    if action == 'RCV_SYN':
        if current_state in ('LISTEN', 'SYN_SENT'):
            return 'SYN_RCVD'
        else:
            return 'ERROR'

    if action == 'APP_CLOSE':
        if current_state in ('LISTEN', 'SYN_SENT'):
            return 'CLOSED'
        elif current_state in ('SYN_RCVD', 'ESTABLISHED'):
            return 'FIN_WAIT_1'
        elif current_state == 'CLOSE_WAIT':
            return 'LAST_ACK'
        else:
            return 'ERROR'

    if current_state == 'FIN_WAIT_1':
        if action == 'RCV_FIN':
            return 'CLOSING'
        elif action == 'RCV_FIN_ACK':
            return 'TIME_WAIT'
        elif action == 'RCV_ACK':
            return 'FIN_WAIT_2'
        else:
            return 'ERROR'

    if action == 'RCV_ACK':
        if current_state == 'SYN_RCVD':
            return 'ESTABLISHED'
        elif current_state == 'CLOSING':
            return 'TIME_WAIT'
        elif current_state == 'LAST_ACK':
            return 'CLOSED'
        else:
            return 'ERROR'

    if action == 'RCV_FIN':
        if current_state == 'ESTABLISHED':
            return 'CLOSE_WAIT'
        elif current_state == 'FIN_WAIT_2':
            return 'TIME_WAIT'
        else:
            return 'ERROR'

    if current_state == 'LISTEN':
        if action == 'APP_SEND':
            return 'SYN_SENT'
        else:
            return 'ERROR'

    if current_state == 'SYN_SENT':
        if action == 'RCV_SYN_ACK':
            return 'ESTABLISHED'
        else:
            return 'ERROR'

    if current_state == 'TIME_WAIT':
        if action == 'APP_TIMEOUT':
            return 'CLOSED'
        else:
            return 'ERROR'

    return 'ERROR'

In [19]:
def traverse_TCP_states(events):
    state = "CLOSED"  # initial state, always
    for entry in events:
        state = get_new_state(state, entry)
        if state == 'ERROR':
            return state
    return state

In [20]:
traverse_TCP_states(["APP_ACTIVE_OPEN","RCV_SYN_ACK","RCV_FIN"])
# CLOSE_WAIT

'CLOSE_WAIT'